## 1. Configuración e Imports

In [1]:
import pandas as pd
import numpy as np
import pickle
import re
from pathlib import Path
from scipy.sparse import hstack, csr_matrix
import warnings
warnings.filterwarnings('ignore')

# Rutas
BASE_DIR = Path('..').resolve()
MODEL_DIR = BASE_DIR / 'models'
DATA_DIR = BASE_DIR / 'data' / 'vectorized'

print(f"✓ Directorio de modelos: {MODEL_DIR}")
print(f"✓ Directorio de datos: {DATA_DIR}")

✓ Directorio de modelos: D:\Diplomatura en ia\trabajo practico 3 -Omar Gonzalez\tp3_nlp_sentiment\models
✓ Directorio de datos: D:\Diplomatura en ia\trabajo practico 3 -Omar Gonzalez\tp3_nlp_sentiment\data\vectorized


## 2. Cargar Modelo y Vectorizador

In [2]:
# Cargar métricas para saber cuál es el mejor modelo
with open(MODEL_DIR / 'model_metrics.pkl', 'rb') as f:
    metrics = pickle.load(f)

model_name = metrics['best_model_name']
print(f"✓ Mejor modelo: {model_name}")

# Cargar el modelo
model_filename = f"best_model_{model_name.replace(' ', '_').lower()}.pkl"
with open(MODEL_DIR / model_filename, 'rb') as f:
    model = pickle.load(f)

print(f"✓ Modelo cargado: {type(model).__name__}")

# Cargar vectorizador TF-IDF
with open(DATA_DIR / 'tfidf_vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

print(f"✓ Vectorizador cargado: {vectorizer.max_features} features")

✓ Mejor modelo: Linear SVM
✓ Modelo cargado: LinearSVC
✓ Vectorizador cargado: 10000 features


## 3. Funciones de Preprocesamiento

Recreamos las mismas funciones del notebook `02_preprocessing.ipynb`:

In [3]:
def preprocess_tweet(text):
    """
    Preprocesa un tweet según las 9 decisiones del EDA.
    """
    if pd.isna(text) or not text:
        return ""
    
    # 1. Eliminar URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    
    # 2. Eliminar mentions (@usuario)
    text = re.sub(r'@\w+', '', text)
    
    # 3. Extraer texto de hashtags (#fail → fail)
    text = re.sub(r'#(\w+)', r'\1', text)
    
    # 4. Normalizar caracteres repetidos (goooood → good)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    
    # 5. Convertir a minúsculas
    text = text.lower()
    
    # 6. Eliminar caracteres especiales (conservar solo letras/números)
    text = re.sub(r'[^a-z0-9\s]', '', text)
    
    # 7. Eliminar espacios múltiples
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text


def extract_features(text):
    """
    Extrae 7 features numéricas del texto ORIGINAL (antes de limpiar).
    """
    if pd.isna(text) or not text:
        return {
            'length': 0,
            'num_words': 0,
            'num_hashtags': 0,
            'num_mentions': 0,
            'num_urls': 0,
            'num_uppercase': 0,
            'pct_uppercase': 0.0
        }
    
    length = len(text)
    num_uppercase = sum(1 for c in text if c.isupper())
    
    return {
        'length': length,
        'num_words': len(text.split()),
        'num_hashtags': len(re.findall(r'#\w+', text)),
        'num_mentions': len(re.findall(r'@\w+', text)),
        'num_urls': len(re.findall(r'http\S+|www\S+', text)),
        'num_uppercase': num_uppercase,
        'pct_uppercase': (num_uppercase / length * 100) if length > 0 else 0
    }

print("✓ Funciones de preprocesamiento definidas")

✓ Funciones de preprocesamiento definidas


## 4. Función de Predicción Completa

In [4]:
def predict_sentiment(text, return_details=False):
    """
    Predice el sentimiento de un tweet.
    
    Args:
        text: Tweet original (string)
        return_details: Si True, retorna dict con detalles
    
    Returns:
        Si return_details=False: 'Positivo' o 'Negativo'
        Si return_details=True: dict con:
            - text: texto original
            - text_clean: texto limpio
            - sentiment: 'Positivo' o 'Negativo'
            - confidence: probabilidad (si disponible)
            - features: features numéricas
    """
    # 1. Extraer features del texto ORIGINAL
    features_dict = extract_features(text)
    
    # 2. Limpiar texto
    text_clean = preprocess_tweet(text)
    
    if not text_clean:
        if return_details:
            return {
                'text': text,
                'text_clean': '',
                'sentiment': 'Neutral',
                'confidence': None,
                'features': features_dict,
                'error': 'Texto vacío después de limpieza'
            }
        return 'Neutral'
    
    # 3. Vectorizar texto con TF-IDF
    X_text = vectorizer.transform([text_clean])
    
    # 4. Agregar features numéricas
    features_array = np.array([[
        features_dict['length'],
        features_dict['num_words'],
        features_dict['num_hashtags'],
        features_dict['num_mentions'],
        features_dict['num_urls'],
        features_dict['num_uppercase'],
        features_dict['pct_uppercase']
    ]])
    
    # 5. Combinar TF-IDF + features numéricas
    X_final = hstack([X_text, csr_matrix(features_array)])
    
    # 6. Predecir
    prediction = model.predict(X_final)[0]
    sentiment = 'Positivo' if prediction == 1 else 'Negativo'
    
    # 7. Calcular confianza (si el modelo lo permite)
    confidence = None
    if hasattr(model, 'predict_proba'):
        proba = model.predict_proba(X_final)[0]
        confidence = float(proba[prediction])
    elif hasattr(model, 'decision_function'):
        score = model.decision_function(X_final)[0]
        # Convertir a probabilidad aproximada usando sigmoide
        confidence = float(1 / (1 + np.exp(-score)))
    
    if return_details:
        return {
            'text': text,
            'text_clean': text_clean,
            'sentiment': sentiment,
            'confidence': confidence,
            'features': features_dict
        }
    
    return sentiment

print("✓ Función predict_sentiment() lista")

✓ Función predict_sentiment() lista


## 5. Cargar y Predecir sobre testdata.manual.2009.06.14.csv

Cargamos el archivo CSV de test y predecimos el sentimiento de cada tweet:

In [5]:
# Cargar el CSV de test
test_csv_path = BASE_DIR / 'data' / 'raw' / 'testdata.manual.2009.06.14.csv'

# Leer el CSV (mismo formato que el training)
df_test = pd.read_csv(
    test_csv_path,
    encoding='latin-1',
    header=None,
    names=['polarity', 'id', 'date', 'query', 'user', 'text']
)

print(f"✓ Archivo cargado: {test_csv_path.name}")
print(f"  Total de tweets: {len(df_test)}")
print(f"\nDistribución original:")
print(df_test['polarity'].value_counts().sort_index())
print(f"  0 = Negativo")
print(f"  2 = Neutral") 
print(f"  4 = Positivo")

✓ Archivo cargado: testdata.manual.2009.06.14.csv
  Total de tweets: 498

Distribución original:
polarity
0    177
2    139
4    182
Name: count, dtype: int64
  0 = Negativo
  2 = Neutral
  4 = Positivo


In [6]:
# Procesar todos los tweets
print("\nProcesando tweets...")

results = []
for idx, row in df_test.iterrows():
    text = row['text']
    result = predict_sentiment(text, return_details=True)
    
    # Agregar datos originales
    result['true_polarity'] = row['polarity']
    result['true_label'] = {0: 'Negativo', 2: 'Neutral', 4: 'Positivo'}[row['polarity']]
    result['predicted_value'] = 1 if result['sentiment'] == 'Positivo' else 0
    
    results.append(result)

# Convertir a DataFrame
df_predictions = pd.DataFrame(results)

print(f"✓ Procesados {len(df_predictions)} tweets")
print(f"\nPrimeras predicciones:")
df_predictions[['text', 'sentiment', 'confidence', 'true_label']].head(10)


Procesando tweets...
✓ Procesados 498 tweets

Primeras predicciones:


,text,sentiment,confidence,true_label
0,@stellargirl I loooooooovvvvvveee my Kindle2. ...,Positivo,0.512669,Positivo
1,Reading my kindle2... Love it... Lee childs i...,Positivo,0.707605,Positivo
2,"Ok, first assesment of the #kindle2 ...it fuck...",Positivo,0.621847,Positivo
3,@kenburbary You'll love your Kindle2. I've had...,Positivo,0.638937,Positivo
4,@mikefish Fair enough. But i have the Kindle2...,Positivo,0.626575,Positivo
5,@richardebaker no. it is too big. I'm quite ha...,Negativo,0.476778,Positivo
6,Fuck this economy. I hate aig and their non lo...,Negativo,0.231442,Negativo
7,Jquery is my new best friend.,Positivo,0.603437,Positivo
8,Loves twitter,Positivo,0.729982,Positivo
9,how can you not love Obama? he makes jokes abo...,Positivo,0.554366,Positivo


## 6. Análisis de Resultados

In [7]:
print("="*80)
print("RESUMEN DE PREDICCIONES")
print("="*80)

# Distribución de predicciones
print(f"\nTOTAL: {len(df_predictions)} tweets")
print(f"\nPredicciones del modelo:")
print(df_predictions['sentiment'].value_counts())
print(f"\n  Positivos: {(df_predictions['sentiment'] == 'Positivo').sum()} ({(df_predictions['sentiment'] == 'Positivo').sum()/len(df_predictions)*100:.1f}%)")
print(f"  Negativos: {(df_predictions['sentiment'] == 'Negativo').sum()} ({(df_predictions['sentiment'] == 'Negativo').sum()/len(df_predictions)*100:.1f}%)")

if df_predictions['confidence'].notna().any():
    print(f"\nConfianza promedio: {df_predictions['confidence'].mean():.1%}")
    print(f"  Mínima: {df_predictions['confidence'].min():.1%}")
    print(f"  Máxima: {df_predictions['confidence'].max():.1%}")

RESUMEN DE PREDICCIONES

TOTAL: 498 tweets

Predicciones del modelo:
sentiment
Positivo    306
Negativo    192
Name: count, dtype: int64

  Positivos: 306 (61.4%)
  Negativos: 192 (38.6%)

Confianza promedio: 52.4%
  Mínima: 11.3%
  Máxima: 89.4%


In [8]:
# Mostrar algunos ejemplos de predicciones
print("\n" + "="*80)
print("EJEMPLOS DE PREDICCIONES")
print("="*80)

# Ejemplos positivos predichos
print("\n📊 Tweets predichos como POSITIVOS (primeros 5):")
positivos = df_predictions[df_predictions['sentiment'] == 'Positivo'].head(5)
for idx, row in positivos.iterrows():
    print(f"\n  Tweet: {row['text'][:80]}...")
    print(f"  Predicción: {row['sentiment']} | Real: {row['true_label']} | Confianza: {row['confidence']:.1%}")

# Ejemplos negativos predichos
print("\n📊 Tweets predichos como NEGATIVOS (primeros 5):")
negativos = df_predictions[df_predictions['sentiment'] == 'Negativo'].head(5)
for idx, row in negativos.iterrows():
    print(f"\n  Tweet: {row['text'][:80]}...")
    print(f"  Predicción: {row['sentiment']} | Real: {row['true_label']} | Confianza: {row['confidence']:.1%}")


EJEMPLOS DE PREDICCIONES

📊 Tweets predichos como POSITIVOS (primeros 5):

  Tweet: @stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2...
  Predicción: Positivo | Real: Positivo | Confianza: 51.3%

  Tweet: Reading my kindle2...  Love it... Lee childs is good read....
  Predicción: Positivo | Real: Positivo | Confianza: 70.8%

  Tweet: Ok, first assesment of the #kindle2 ...it fucking rocks!!!...
  Predicción: Positivo | Real: Positivo | Confianza: 62.2%

  Tweet: @kenburbary You'll love your Kindle2. I've had mine for a few months and never l...
  Predicción: Positivo | Real: Positivo | Confianza: 63.9%

  Tweet: @mikefish  Fair enough. But i have the Kindle2 and I think it's perfect  :)...
  Predicción: Positivo | Real: Positivo | Confianza: 62.7%

📊 Tweets predichos como NEGATIVOS (primeros 5):

  Tweet: @richardebaker no. it is too big. I'm quite happy with the Kindle2....
  Predicción: Negativo | Real: Positivo | Confianza: 47.7%

  Tweet: Fuck this 

In [9]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score

# ============================================================================
# MÉTRICAS FORMALES EN TEST SET (Solo tweets binarios)
# ============================================================================

# Filtrar solo tweets binarios (excluyendo neutros con polarity=2)
df_binary = df_predictions[df_predictions['true_polarity'] != 2].copy()

print("=" * 80)
print("MÉTRICAS FORMALES EN TEST SET")
print("=" * 80)
print(f"\n Tweets evaluados: {len(df_binary)} (excluidos {len(df_predictions) - len(df_binary)} neutros)")

# Preparar datos para sklearn
y_true = df_binary['true_polarity'].map({0: 0, 4: 1})
y_pred = df_binary['predicted_value']

# Métricas principales
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"\n 📊 MÉTRICAS PRINCIPALES:")
print(f"   Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"   F1-Score:  {f1:.4f} ({f1*100:.2f}%)")

# Classification report
print(f"\n 📋 CLASSIFICATION REPORT:")
print(classification_report(y_true, y_pred, target_names=['Negativo', 'Positivo']))

# Matriz de confusión
cm = confusion_matrix(y_true, y_pred)
print(f"\n 🔲 MATRIZ DE CONFUSIÓN:")
print(f"                Predicción")
print(f"              Neg     Pos")
print(f"Real Neg     {cm[0,0]:4d}    {cm[0,1]:4d}")
print(f"Real Pos     {cm[1,0]:4d}    {cm[1,1]:4d}")

# Interpretación
tn, fp, fn, tp = cm.ravel()
print(f"\n 📈 INTERPRETACIÓN:")
print(f"   True Negatives:  {tn} (negativos correctos)")
print(f"   True Positives:  {tp} (positivos correctos)")
print(f"   False Positives: {fp} (predichos pos, eran neg)")
print(f"   False Negatives: {fn} (predichos neg, eran pos)")

MÉTRICAS FORMALES EN TEST SET

 Tweets evaluados: 359 (excluidos 139 neutros)

 📊 MÉTRICAS PRINCIPALES:
   Accuracy:  0.8468 (84.68%)
   F1-Score:  0.8518 (85.18%)

 📋 CLASSIFICATION REPORT:
              precision    recall  f1-score   support

    Negativo       0.86      0.82      0.84       177
    Positivo       0.84      0.87      0.85       182

    accuracy                           0.85       359
   macro avg       0.85      0.85      0.85       359
weighted avg       0.85      0.85      0.85       359


 🔲 MATRIZ DE CONFUSIÓN:
                Predicción
              Neg     Pos
Real Neg      146      31
Real Pos       24     158

 📈 INTERPRETACIÓN:
   True Negatives:  146 (negativos correctos)
   True Positives:  158 (positivos correctos)
   False Positives: 31 (predichos pos, eran neg)
   False Negatives: 24 (predichos neg, eran pos)


In [10]:
# ============================================================================
# ANÁLISIS DE ERRORES
# ============================================================================

# Identificar errores
df_binary['correct'] = (df_binary['predicted_value'] == y_true)
df_errors = df_binary[~df_binary['correct']]

print("\n" + "=" * 80)
print("ANÁLISIS DE ERRORES")
print("=" * 80)
print(f"\n Total errores: {len(df_errors)} de {len(df_binary)} ({len(df_errors)/len(df_binary)*100:.1f}%)")

# Falsos Positivos (predicho positivo, era negativo)
fp_df = df_errors[(df_errors['predicted_value'] == 1) & (df_errors['true_polarity'] == 0)]

print(f"\n ⚠️ FALSOS POSITIVOS: {len(fp_df)} casos")
print("   (Predicho: Positivo → Real: Negativo)")
if len(fp_df) > 0:
    print("\n    Ejemplos:")
    for i, row in fp_df.head(5).iterrows():
        print(f"   • '{row['text'][:70]}...'")
        print(f"     Confianza: {row['confidence']:.1%}")

# Falsos Negativos (predicho negativo, era positivo)
fn_df = df_errors[(df_errors['predicted_value'] == 0) & (df_errors['true_polarity'] == 4)]

print(f"\n ⚠️ FALSOS NEGATIVOS: {len(fn_df)} casos")
print("   (Predicho: Negativo → Real: Positivo)")
if len(fn_df) > 0:
    print("\n    Ejemplos:")
    for i, row in fn_df.head(5).iterrows():
        print(f"   • '{row['text'][:70]}...'")
        print(f"     Confianza: {row['confidence']:.1%}")


ANÁLISIS DE ERRORES

 Total errores: 55 de 359 (15.3%)

 ⚠️ FALSOS POSITIVOS: 31 casos
   (Predicho: Positivo → Real: Negativo)

    Ejemplos:
   • 'US planning to resume the military tribunals at Guantanamo Bay... only...'
     Confianza: 64.4%
   • '?Obama Administration Must Stop Bonuses to AIG Ponzi Schemers ... http...'
     Confianza: 56.4%
   • 'ShaunWoo hate'n on AiG...'
     Confianza: 52.8%
   • 'yahoo answers can be a butt sometimes...'
     Confianza: 50.5%
   • '@legalgeekery Yeahhhhhhhhh, I wouldn't really have lived in East Palo ...'
     Confianza: 51.6%

 ⚠️ FALSOS NEGATIVOS: 24 casos
   (Predicho: Negativo → Real: Positivo)

    Ejemplos:
   • '@richardebaker no. it is too big. I'm quite happy with the Kindle2....'
     Confianza: 47.7%
   • '#lebron best athlete of our generation, if not all time (basketball re...'
     Confianza: 45.0%
   • '@wordwhizkid Lebron is a beast... nobody in the NBA comes even close....'
     Confianza: 46.1%
   • '@SoChi2 I current use t

## 7. Guardar Resultados en CSV

Guardamos las predicciones completas:

In [11]:
# Preparar DataFrame final con las columnas importantes
df_output = pd.DataFrame({
    'text': df_predictions['text'],
    'text_clean': df_predictions['text_clean'],
    'predicted_sentiment': df_predictions['sentiment'],
    'confidence': df_predictions['confidence'],
    'true_polarity': df_predictions['true_polarity'],
    'true_label': df_predictions['true_label'],
    'length': [f['length'] for f in df_predictions['features']],
    'num_words': [f['num_words'] for f in df_predictions['features']],
    'num_hashtags': [f['num_hashtags'] for f in df_predictions['features']],
    'num_mentions': [f['num_mentions'] for f in df_predictions['features']]
})

# Guardar en CSV
output_path = BASE_DIR / 'data' / 'predictions' / 'testdata_predictions.csv'
output_path.parent.mkdir(parents=True, exist_ok=True)

df_output.to_csv(output_path, index=False, encoding='utf-8')
print(f"✓ Predicciones guardadas en: {output_path}")
print(f"  Total de registros: {len(df_output)}")
print(f"\nColumnas guardadas:")
for col in df_output.columns:
    print(f"  - {col}")

✓ Predicciones guardadas en: D:\Diplomatura en ia\trabajo practico 3 -Omar Gonzalez\tp3_nlp_sentiment\data\predictions\testdata_predictions.csv
  Total de registros: 498

Columnas guardadas:
  - text
  - text_clean
  - predicted_sentiment
  - confidence
  - true_polarity
  - true_label
  - length
  - num_words
  - num_hashtags
  - num_mentions


## 8. Script de Producción

script Python completo en `../predict_sentiment.py` que incluye:

### Uso desde línea de comandos:

```bash
# Predicción de un solo tweet|
python predict_sentiment.py "I love this product!"

# Predicción batch desde archivo
python predict_sentiment.py --file tweets.txt --output results.csv

# Con información detallada
python predict_sentiment.py "Great day!" --verbose
```

### Uso desde código Python:

```python
from predict_sentiment import SentimentPredictor

# Crear predictor
predictor = SentimentPredictor()

# Predecir un tweet
result = predictor.predict_single("I love this!")
print(result['sentiment'], result['confidence'])

# Predecir varios tweets
results = predictor.predict_batch(["tweet1", "tweet2", "tweet3"])
```

## 9. Prueba del Script

Verificamos que el script funciona correctamente:

In [12]:
# Importar la clase del script
import sys
sys.path.append(str(BASE_DIR))

from predict_sentiment import SentimentPredictor

# Crear predictor
predictor = SentimentPredictor(
    model_dir=str(MODEL_DIR),
    data_dir=str(DATA_DIR)
)

print("\n" + "="*80)
print("PRUEBA DEL SCRIPT DE PRODUCCIÓN")
print("="*80)

# Probar con ejemplos
ejemplos = [
    "This product is amazing! I love it!",
    "Worst purchase ever. Total waste of money.",
    "Just okay, nothing special."
]

for tweet in ejemplos:
    result = predictor.predict_single(tweet)
    print(f"\n{tweet}")
    print(f"→ {result['sentiment']} (confianza: {result['confidence']:.1%})")

✓ Modelo cargado: Linear SVM
✓ Vectorizador cargado

PRUEBA DEL SCRIPT DE PRODUCCIÓN

This product is amazing! I love it!
→ Positivo (confianza: 71.8%)

Worst purchase ever. Total waste of money.
→ Negativo (confianza: 30.7%)

Just okay, nothing special.
→ Positivo (confianza: 53.2%)


---

## 📊 Conclusiones

### ✅ Pipeline de Producción Completo

1. **Modelo listo para usar:**
   - Cargado desde disco (`models/best_model_*.pkl`)
   - Vectorizador TF-IDF con bigramas incluido
   - Preprocessing automático integrado

2. **Múltiples formas de uso:**
   - Notebook interactivo (este archivo)
   - Script de línea de comandos (`predict_sentiment.py`)
   - Clase Python reutilizable (`SentimentPredictor`)

3. **Funcionalidades:**
   - Predicción de tweets individuales
   - Predicción batch de múltiples tweets
   - Cálculo de confianza (probabilidad)
   - Extracción de features numéricas
   - Exportación a CSV

4. **Listo para integrar en:**
   - API REST (Flask/FastAPI)
   - Aplicación web
   - Pipeline de datos en tiempo real
   - Sistema de monitoreo de redes sociales

---
